# Test 7
With ETA

In [18]:
import pandas as pd
import xgboost as xgb

In [19]:
train = pd.read_csv('ais_train.csv', sep='|')
train['time'] = pd.to_datetime(train['time'])
print(train.head()) 

test = pd.read_csv('ais_test.csv', sep=',')
test['time'] = pd.to_datetime(test['time'])

vessels = pd.read_csv('vessels.csv', sep='|')

ports = pd.read_csv('ports.csv', sep='|')

                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-01 00:00:25  284.0   0.7    0       88        0  01-09 23:00   
1 2024-01-01 00:00:36  109.6   0.0   -6      347        1  12-29 20:00   
2 2024-01-01 00:01:45  111.0  11.0    0      112        0  01-02 09:00   
3 2024-01-01 00:03:11   96.4   0.0    0      142        1  12-31 20:00   
4 2024-01-01 00:03:51  214.0  19.7    0      215        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f  
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689  
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19  
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126  
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3  


In [20]:
# Mappings for IDs
port_id_mapping = {port_id: idx for idx, port_id in enumerate(train['portId'].unique())}
train['portId'] = train['portId'].map(port_id_mapping)
ports['portId'] = ports['portId'].map(port_id_mapping)

vessel_id_mapping = {vessel_id: idx for idx, vessel_id in enumerate(train['vesselId'].unique())}
train['vesselId'] = train['vesselId'].map(vessel_id_mapping)
vessels['vesselId'] = vessels['vesselId'].map(vessel_id_mapping)
test['vesselId'] = test['vesselId'].map(vessel_id_mapping)

shipping_line_id_mapping = {shipping_line_id: idx for idx, shipping_line_id in enumerate(vessels['shippingLineId'].unique())}
vessels['shippingLineId'] = vessels['shippingLineId'].map(shipping_line_id_mapping)

In [31]:
import pandas as pd
from datetime import timedelta

# Function to convert 'etaRaw' to datetime format, handling year rollovers
def convert_eta_raw(row):
    current_year = row['time'].year  # Get the year from the 'time' column
    eta_with_year = f"{current_year}-{row['etaRaw']}"  # Create full datetime string
    eta_datetime = pd.to_datetime(eta_with_year, format='%Y-%m-%d %H:%M', errors='coerce')
    
    # Adjust year if 'etaRaw' is earlier than 'time'
    if eta_datetime < row['time']:
        eta_datetime = eta_datetime + timedelta(days=365)
    return eta_datetime

def feature_engineering_heading(test, train):
    test = test.copy()
    train = train.copy()
    features = pd.DataFrame()

    # Add the columns vesselId, time, latitude, longitude, heading and sog to the features from train
    features['vesselId'] = train['vesselId']
    features['time'] = train['time']
    features['etaRaw'] = train['etaRaw']
    features['latitude'] = train['latitude']
    features['longitude'] = train['longitude']
    features['cog'] = train['cog']
    features['sog'] = train['sog']

    # Create a new binary column for 'under_way' based on the navstat column in train. 
    # 1 if navstat is 0 or 8, 0 otherwise
    features['under_way'] = train['navstat'].isin([0, 8]).astype(int)

    # Adding the columns latitude_1 (last seen latitude) and longitude_1 (last senn longitude)
    # to the features from train. For each vessel
    # Also adding time_1 (time difference from current time to last seen time)
    features['latitude_1'] = train.groupby('vesselId')['latitude'].shift(1)
    features['longitude_1'] = train.groupby('vesselId')['longitude'].shift(1)
    features['time_1'] = train.groupby('vesselId')['time'].diff().dt.total_seconds()

    # Adding the columns latitude_2 (second last seen latitude) and longitude_2 (second last seen longitude)
    # to the features from train. For each vessel
    # Also adding time_2 (time difference from current time to second last seen time)
    features['latitude_2'] = train.groupby('vesselId')['latitude'].shift(2)
    features['longitude_2'] = train.groupby('vesselId')['longitude'].shift(2)
    features['time_2'] = train.groupby('vesselId')['time'].diff(2).dt.total_seconds()

    # Adding the columns latitude_3 (third last seen latitude) and longitude_3 (third last seen longitude)
    # to the features from train. For each vessel
    # Also adding time_3 (time difference from current time to third last seen time)
    features['latitude_3'] = train.groupby('vesselId')['latitude'].shift(3)
    features['longitude_3'] = train.groupby('vesselId')['longitude'].shift(3)
    features['time_3'] = train.groupby('vesselId')['time'].diff(3).dt.total_seconds()


    # Apply the conversion function to create a new 'eta' column
    features['eta'] = features.apply(convert_eta_raw, axis=1)

    # Calculate the time difference between 'time' and 'eta' in seconds
    features['time_difference_eta'] = (features['eta'] - features['time']).dt.total_seconds()

    # Dropping the rows with NaN values
    features = features.dropna()

    # Split the time column into month, day, hour, minute, and second columns
    features['month'] = train['time'].dt.month
    features['day'] = train['time'].dt.day
    features['hour'] = train['time'].dt.hour
    features['minute'] = train['time'].dt.minute

    # Drop the 'time' and 'etaRaw' columns as they are no longer needed
    features.drop(['time', 'etaRaw'], axis=1, inplace=True)
    features.drop(['eta'], axis=1, inplace=True)

    return features

# Example usage
features_heading = feature_engineering_heading(test, train)
print(features_heading.head())
print(features_heading.shape)


     vesselId  latitude  longitude    cog   sog  under_way  latitude_1  \
439        87  51.46019    2.69617  329.1   1.1          0    51.45925   
442        21  18.94058  -66.48705  318.6  12.5          1    18.91427   
443        28  55.46484   14.91138   60.9  16.7          1    55.44269   
445        82  40.63020   18.28014  316.8  11.0          1    40.59254   
449        25  45.55134   13.73574  267.7   0.0          0    45.55135   

     longitude_1  time_1  latitude_2  longitude_2  time_2  latitude_3  \
439      2.69635   357.0    51.45980      2.69436  2342.0    51.45926   
442    -66.46253   600.0    18.85826    -66.41107  1880.0    18.80632   
443     14.84121   588.0    55.39951     14.71121  1680.0    55.33025   
445     18.32469  1008.0    40.56030     18.34426  1708.0    40.50311   
449     13.73573   904.0    45.55136     13.73578  2520.0    45.55133   

     longitude_3  time_3  time_difference_eta  month  day  hour  minute  
439      2.69441  2703.0           3156115

In [32]:
def find_last_features(features):
    # Assuming that the last row for every vesselId is the most recent
    last_features = features.groupby('vesselId').last().reset_index()
    return last_features

last_features_heading = find_last_features(features_heading)
print(last_features_heading.head())
print(last_features_heading.shape)

   vesselId  latitude  longitude    cog   sog  under_way  latitude_1  \
0         0  34.57936  128.99926  221.5  15.5          1    34.59684   
1         1   1.24460  103.39997  305.1  15.7          1     1.22186   
2         2  18.13873  -69.74863  176.0   0.4          1    18.14185   
3         3  41.64055  143.29942   87.6  14.4          1    41.63254   
4         4  26.58710  121.27831   39.1  12.7          1    26.54636   

   longitude_1   time_1  latitude_2  longitude_2   time_2  latitude_3  \
0    129.01917    335.0    34.65578    129.09349   1535.0    34.71258   
1    103.46804   1007.0     1.20238    103.55412   2220.0     1.18040   
2    -69.74807   1250.0    18.14458    -69.74653   2470.0    18.14640   
3    141.92751  14489.0    41.63043    141.86587  15126.0    41.62621   
4    121.23948    908.0    26.49491    121.19909   1970.0    26.34699   

   longitude_3   time_3  time_difference_eta  month  day  hour  minute  
0    129.17443   2795.0              83883.0      5    

In [33]:
# Define features and target of heading model
y = features_heading[['latitude', 'longitude']]  
X = features_heading.drop(columns=['latitude', 'longitude']) 

In [34]:
# Initialize the xgboost model
model_heading = xgb.XGBRegressor()

# Fit the model
model_heading.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [35]:
### Prepare test data for predictions
def prepare_test_for_predictions(test, last_features):
    test = test.copy()
    prepared_test = pd.DataFrame()

    # Create a time column in last features
    last_features['year'] = 2024
    last_features['time'] = pd.to_datetime(
        last_features[['year', 'month', 'day', 'hour', 'minute']]
    )
    last_features =last_features.drop(columns=['longitude_3', 'latitude_3', 'year', 'month', 'day', 'hour', 'minute', 'time_3']).copy()


    # Add the columns vesselId and time
    prepared_test['vesselId'] = test['vesselId']
    prepared_test['time'] = test['time']

    # For each vessel, add the last seen features to the prepared test
    prepared_test = prepared_test.merge(last_features, on='vesselId', how='left', suffixes=('', '_last'))

    # Add the empty columns latitude_3, longitude_3 and time_3 to the prepared test
    prepared_test['latitude_3'] = None
    prepared_test['longitude_3'] = None
    prepared_test['time_3'] = None

    # For each row, latitude_3 should be the same as latitude_2, longitude_3 should be the same as longitude_2
    # and time_3 should be the same as time_2
    prepared_test['latitude_3'] = prepared_test['latitude_2']
    prepared_test['longitude_3'] = prepared_test['longitude_2']
    prepared_test['time_3'] = prepared_test['time_2']

    # Same for latitude_2, longitude_2 and time_2
    prepared_test['latitude_2'] = prepared_test['latitude_1']
    prepared_test['longitude_2'] = prepared_test['longitude_1']
    prepared_test['time_2'] = prepared_test['time_1']

    # Same for latitude_1, longitude_1 and time_1
    prepared_test['latitude_1'] = prepared_test['latitude']
    prepared_test['longitude_1'] = prepared_test['longitude']

    # Time_1 should be the difference between the current time and the last seen time
    prepared_test['time_1'] = (prepared_test['time'] - prepared_test['time_last']).dt.total_seconds()

    # For each time column (time_2, time_3), add the rows value in time_1
    prepared_test['time_2'] += prepared_test['time_1']
    prepared_test['time_3'] += prepared_test['time_1']

    # Subtract time_1 from difference_to_eta
    prepared_test['time_difference_eta'] = (prepared_test['time_difference_eta'] - prepared_test['time_1'])

    # Drop the the latitude and longitude columns
    prepared_test.drop(columns=['latitude', 'longitude'], inplace=True)
    print(prepared_test.shape)

    # Split the time column into month, day, hour, minute and second columns
    prepared_test['month'] = test['time'].dt.month
    prepared_test['day'] = test['time'].dt.day
    prepared_test['hour'] = test['time'].dt.hour
    prepared_test['minute'] = test['time'].dt.minute
    # prepared_test['second'] = test['time'].dt.second

    prepared_test.drop('time', axis=1, inplace=True)
    prepared_test.drop('time_last', axis=1, inplace=True)

    return prepared_test

test_df = prepare_test_for_predictions(test, last_features_heading)
print(test_df.head())
print(test_df.shape)


(51739, 16)
   vesselId    cog   sog  under_way  latitude_1  longitude_1  time_1  \
0       412  179.6   0.0          0    31.14647    -81.49789   916.0   
1       373   24.7   0.0          0    14.81694    120.29625   557.0   
2       181    8.0  18.7          1    38.27895     10.78280   662.0   
3         8  321.3   0.1          0   -43.53785    172.83522  1114.0   
4        65  291.0   0.3          0    48.53320     -6.12003  1287.0   

   latitude_2  longitude_2  time_2  time_difference_eta  latitude_3  \
0    31.14648    -81.49789  2172.0           31401688.0    31.14648   
1    14.81694    120.29624  2319.0           31013607.0    14.81688   
2    38.14875     10.75635  2168.0              45290.0    36.81120   
3   -43.53815    172.83516  2014.0           31453432.0   -43.53800   
4    48.53133     -6.10750  3260.0             100024.0    48.53133   

   longitude_3   time_3  month  day  hour  minute  
0    -81.49789   2896.0      5    8     0       3  
1    120.29630   4123.0 

In [37]:
test_df = test_df[['vesselId', 'cog', 'sog', 'under_way', 'latitude_1', 'longitude_1', 'time_1', 'latitude_2', 'longitude_2', 'time_2', 'latitude_3', 'longitude_3', 'time_3', 'time_difference_eta', 'month', 'day', 'hour', 'minute']]

In [38]:
# Make predictions
predictions = model_heading.predict(test_df)
print(predictions)
print(predictions.shape)

[[  31.329191  -81.336815]
 [  14.33112   118.91559 ]
 [  38.38925    10.929755]
 ...
 [  43.609764 -113.26756 ]
 [  53.00501    28.998825]
 [  62.139797   17.509064]]
(51739, 2)


In [39]:
# Create a DataFrame with the required format
predictions_df = pd.DataFrame(predictions, columns=['latitude_predicted', 'longitude_predicted'])
predictions_df['ID'] = range(len(predictions_df))
predictions_df = predictions_df[['ID', 'longitude_predicted', 'latitude_predicted']]

# Save to CSV
predictions_df.to_csv('predictions_7.csv', index=False)